[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД


In [28]:
import requests
import os
import csv
import pandas as pd
from tqdm.auto import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse

import warnings
warnings.filterwarnings('ignore')

## Список объектов

In [5]:
# скачиваем информацию о всех строящихся объектах на сайте "наш.дом.рф" в pandas dataframe

offset = 0
df = pd.DataFrame()

while True:
        
        url = 'https://наш.дом.рф/сервисы/api/kn/object'
        params = (
                  ('offset', offset),
                  ('limit', 100),
                  ('sortField', 'devId.devShortCleanNm'),
                  ('sortType', 'asc'),
                  ('objStatus', '0')
                 )
         
        response = requests.get(url, params = params)
        reestr = response.json()
        obj = reestr.get('data').get('list')
         
        if not obj:
                   break
     
        df1 = pd.DataFrame(obj)
        df1['developer'] = df1['developer'].apply(lambda x: x['shortName'])
        df = df.append(df1, ignore_index=True)
        
        offset += 100

In [6]:
df

,objId,developer,rpdRegionCd,objAddr,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,objProblemFlg,latitude,...,objGuarantyEscrowFlg,residentialBuildings,rpdNum,pdId,buildType,objStatus,objFloorMin,objFloorMax,objCommercNm,complexShortName
0,29679,212-2,61,г Таганрог,18,2020-03-31,1519.00,0,0,47.2238,...,0,1,61-000818,124647,Жилое,0,NaN,NaN,NaN,NaN
1,13383,2МЕН ГРУПП,72,г Тюмень,68,2022-12-31,6293.78,1,0,57.1519,...,0,1,72-000095,28105,Жилое,0,4.0,20.0,NaN,NaN
2,13397,2МЕН ГРУПП,72,"г Тюмень, ул Профсоюзная, д. 56",65,2024-12-31,6248.51,0,0,57.1519,...,0,1,72-000097,28141,Жилое,0,6.0,12.0,NaN,NaN
3,13399,2МЕН ГРУПП,72,"г Тюмень, ул Профсоюзная, д. 56",0,2024-12-31,0.00,0,0,57.1519,...,0,0,72-000097,28141,Нежилое,0,1.0,1.0,NaN,NaN
4,13398,2МЕН ГРУПП,72,"г Тюмень, ул Профсоюзная, д. 56",43,2024-12-31,4812.98,0,0,57.1519,...,0,1,72-000097,28141,Жилое,0,7.0,10.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10578,28072,ЯРКИЙ МИР,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",312,2024-06-30,14788.80,1,0,55.6591,...,0,1,50-001971,96379,Жилое,0,2.0,26.0,"Жилой комплекс""Томилинский бульвар""",NaN
10579,28071,ЯРКИЙ МИР,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",288,2024-06-30,14870.40,1,0,55.6603,...,0,1,50-002025,99284,Жилое,0,26.0,26.0,"Жилой комплекс""Томилинский бульвар""",NaN
10580,28073,ЯРКИЙ МИР,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",312,2024-06-30,14872.80,1,0,55.6611,...,0,1,50-001972,96446,Жилое,0,26.0,26.0,"Жилой комплекс""Томилинский бульвар""",NaN
10581,44447,ЯРОСЛАВЛЬЗАКАЗЧИК,76,"г Ярославль, Фрунзенский район, территория в р...",127,2022-12-31,6764.22,0,0,57.5679,...,1,1,76-000546,475093,Жилое,0,6.0,6.0,NaN,NaN


In [7]:
# Cохраняем pandas dataframe в excel
df.to_csv('nash-dom-rf.csv')
# Cохраняем pandas dataframe в excel
df.to_excel('nash-dom-rf.xlsx')
# Cохраняем pandas dataframe pickle
df.to_pickle('nash-dom-rf.pkl')

In [9]:
# Cохраняем pandas dataframe в БД
import sqlite3
conn = sqlite3.connect('BDnashdom.db')  
c = conn.cursor()
df.to_sql('nash-dom', conn)

10583

# Данные по определенному объекту

In [12]:
# Для скачивания данных по определенному объекту, нужно добавить его id в конец строки запроса
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/29679'

In [17]:
# скачиваем информацию о всех строящихся объектах на сайте "наш.дом.рф" в pandas dataframe
df2 = pd.DataFrame()
for id in tqdm(df.objId):

    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{id}'
    res = requests.get(url)
    r = res.json()
    df1 = pd.json_normalize(r)
    df2 = df2.append(df1, ignore_index=True)

  0%|          | 0/10583 [00:00<?, ?it/s]

In [18]:
df2

,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,data.developer.comissObjCnt,...,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm,data.metro.id,data.metro.name,data.metro.line,data.metro.color,data.metro.time,data.metro.isWalk,data.metro.colors
0,0,29679,124647,7424,212-2,212-2,212-2,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,13383,28105,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,13397,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,13399,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,13398,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10578,0,28072,96379,7344,ЯРКИЙ МИР,ЯРКИЙ МИР,ЯРКИЙ МИР,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10579,0,28071,99284,7344,ЯРКИЙ МИР,ЯРКИЙ МИР,ЯРКИЙ МИР,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10580,0,28073,96446,7344,ЯРКИЙ МИР,ЯРКИЙ МИР,ЯРКИЙ МИР,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10581,0,44447,475093,2538,ЯРОСЛАВЛЬЗАКАЗЧИК,ЯРОСЛАВЛЬЗАКАЗЧИК,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ЯРОСЛАВЛЬЗАКАЗЧИК,0,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Cохраняем pandas dataframe в excel
df2.to_csv('nash-dom-rf1.csv')
# Cохраняем pandas dataframe в excel
df2.to_excel('nash-dom-rf1.xlsx')
# Cохраняем pandas dataframe pickle
df2.to_pickle('nash-dom-rf1.pkl')

In [24]:
# Cохраняем pandas dataframe в БД
import sqlite3
conn1 = sqlite3.connect('BD.db')  
c1 = conn1.cursor()
df.to_sql('nash-dom2', conn1)

10583

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"

In [29]:
# скачиваем фотографии всех строящихся объектов с сайта "наш.дом.рф"
# источник: https://waksoft.susu.ru/2021/11/03/kak-ispolzuya-python-zagruzit-vse-izobrazheniya-s-veb-straniczy/

def is_valid(url):
    """
    Проверяем, является ли url действительным URL
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_images(url):
    """
    Возвращает все URL‑адреса изображений по одному `url`
    """
    soup = bs(requests.get(url).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Получено изображение"):
        img_url = img.attrs.get("src")
        if not img_url:
            # если img не содержит атрибута src, просто пропускаем
            continue
        # сделаем URL абсолютным, присоединив имя домена к только что извлеченному URL
        img_url = urljoin(url, img_url)
        # удалим URL‑адреса типа '/hsts-pixel.gif?c=3.2.5'
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # наконец, если URL действителен
        if is_valid(img_url):
            urls.append(img_url)
    return urls

def download(url, pathname):
    """
    Загружает файл по URL‑адресу и помещает его в папку `pathname`
    """
    # если путь не существует, создать dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # загружаем тело ответа по частям, а не сразу
    response = requests.get(url, stream=True)
    # получить общий размер файла
    file_size = int(response.headers.get("Content-Length", 0))
    # получаем имя файла
    filename = os.path.join(pathname, url.split("/")[-1])
    # индикатор выполнения, изменение единицы измерения на байты вместо итераций (по умолчанию tqdm)
    progress = tqdm(response.iter_content(1024), f"Загружен {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # записываем прочитанные данные в файл
            f.write(data)
            # обновление индикатора выполнения вручную
            progress.update(len(data))

def main(url, path):
    # получить все изображения
    imgs = get_all_images(url)
    for img in imgs:
        # скачать для каждого img
        download(img, path)

In [36]:
# скачаем первые 10 фото объектов
for id in tqdm(df.objId[0:10]):
    url = f'https://наш.дом.рф/сервисы/каталог-новостроек/объект/{id}'
    main(url, "nash-dom")

  0%|          | 0/10 [00:00<?, ?it/s]

Получено изображение:   0%|          | 0/28 [00:00<?, ?it/s]

Загружен nash-dom\47282070:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Получено изображение:   0%|          | 0/68 [00:00<?, ?it/s]

Загружен nash-dom\47282070:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

...

Загружен nash-dom\eab2d676-02b4-46fe-bd2b-a52a99247b70:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

Загружен nash-dom\46f068fa-d225-419c-9673-95b85173e03c:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Загружен nash-dom\59d7374e-8509-437b-9d2e-ffa8e3205845:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

Загружен nash-dom\6817f548-b064-45c6-9944-2b6ec4224b9c:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Загружен nash-dom\557931d7-76f5-4fb5-a3e1-64b6ad30eece:   0%|          | 0.00/787k [00:00<?, ?B/s]